In [1]:
# TODO: should we save every output as a [geoparquet](https://geoparquet.org/) in the future to improve read performance (reduction 30% read time)?

In [ ]:
%load_ext autoreload
%autoreload 2

In [2]:
from logging import getLogger
import shutil
from pathlib import Path
import geopandas as gpd
import pandas as pd
import requests
import dotenv  

dotenv.load_dotenv()

scripts_dir = Path(".").joinpath("src")
import sys
if scripts_dir not in sys.path:
    sys.path.insert(0, scripts_dir.resolve().as_posix())

from helpers.utils import downloadFile, rm_tree, make_archive, writeReadGCP
from helpers.settings import get_settings
from helpers.file_handler import FileConventionHandler
from pipelines.utils import watch
from pipelines.processors import (
    set_wdpa_id,
    protection_level,
    status,
    create_year,
    calculate_area,
    get_mpas,
    set_location_iso,
    set_fps_classes,
    filter_by_methodology,
    filter_by_terrestrial,
    transform_points,
    clean_geometries,
    simplify_async,
)

In [3]:
mysettings = get_settings()

### eez_intermediate

In [13]:
# Pipe params
force_clean = True
step = "preprocess"
pipe = "eez"

In [14]:
# Data sources
## EEZ
EEZ_url = "https://www.marineregions.org/download_file.php"
EEZ_file_name = "eez_v11.shp"
EEZ_params = {"name": "World_EEZ_v11_20191118.zip"}
EEZ_headers = {
    "content-type": "application/x-www-form-urlencoded",
    "cookie": "PHPSESSID=29190501b4503e4b33725cd6bd01e2c6; vliz_webc=vliz_webc2; jwplayer.captionLabel=Off",
    "dnt": "1",
    "origin": "https://www.marineregions.org",
    "sec-fetch-dest": "document",
    "sec-fetch-mode": "navigate",
    "sec-fetch-site": "same-origin",
    "sec-fetch-user": "?1",
    "upgrade-insecure-requests": "1",
}

EEZ_body = {
    "name": "Jason",
    "organisation": "skytruth",
    "email": "hello@skytruth.com",
    "country": "Spain",
    "user_category": "academia",
    "purpose_category": "Conservation",
    "agree": "1",
}

## High seas
hs_url = "https://www.marineregions.org/download_file.php"
hs_file_name = "High_seas_v1.shp"
hs_params = {"name": "World_High_Seas_v1_20200826.zip"}
hs_headers = {
    "content-type": "application/x-www-form-urlencoded",
    "cookie": "PHPSESSID=29190501b4503e4b33725cd6bd01e2c6; vliz_webc=vliz_webc2; jwplayer.captionLabel=Off",
    "dnt": "1",
    "origin": "https://www.marineregions.org",
    "sec-fetch-dest": "document",
    "sec-fetch-mode": "navigate",
    "sec-fetch-site": "same-origin",
    "sec-fetch-user": "?1",
    "upgrade-insecure-requests": "1",
}
hs_body = {
    "name": "Jason",
    "organisation": "skytruth",
    "email": "hello@skytruth.com",
    "country": "Spain",
    "user_category": "academia",
    "purpose_category": "Conservation",
    "agree": "1",
}

In [15]:
working_folder = FileConventionHandler(pipe)
input_path = working_folder.pipe_raw_path
temp_working_path = working_folder.get_temp_file_path(step)

output_path = working_folder.get_processed_step_path(step)
output_file = working_folder.get_step_fmt_file_path(step, "shp")
zipped_output_file = working_folder.get_step_fmt_file_path(step, "zip", True)
remote_path = working_folder.get_remote_path(step)

In [ ]:
# Extract data
## download files EEZ & High seas
downloadFile(
    EEZ_url,
    input_path,
    EEZ_body,
    EEZ_params,
    EEZ_headers,
    overwrite=force_clean,
)
downloadFile(hs_url, input_path, hs_body, hs_params, hs_headers, overwrite=force_clean)

In [ ]:
## unzip file if needed & load data
unziped_folders = []
for idx, path in enumerate(input_path.glob("*.zip")):
    unziped_folder = temp_working_path.joinpath(path.stem)
    print(unziped_folder)

    if unziped_folder.exists() and force_clean:
        rm_tree(unziped_folder)

    shutil.unpack_archive(path, unziped_folder)

    files = [gpd.read_file(file) for file in unziped_folder.rglob("*.shp") if "boundaries" not in file.stem]
    unziped_folders.append(
        pd.concat(files)
    )

In [ ]:
for idx, gdf in enumerate(unziped_folders):
    print(f"GeoDataFrame {idx} has {len(gdf)} rows and {len(gdf.columns)} columns")

In [73]:
# Transform data
## set the same structure for both datasets updating the high seas one
unziped_folders[0] = (
    unziped_folders[0]
    .rename(
        columns={"name": "GEONAME", "area_km2": "AREA_KM2", "mrgid": "MRGID"},
    )
    .assign(
        POL_TYPE="High Seas",
        ISO_SOV1="ABNJ",
    )
)

# merge datasets
df = pd.concat(unziped_folders, ignore_index=True)

df.drop(
    columns=list(
        set(df.columns)
        - set(
            [
                "MRGID",
                "GEONAME",
                "POL_TYPE",
                "ISO_SOV1",
                "ISO_SOV2",
                "ISO_SOV3",
                "AREA_KM2",
                "geometry",
            ]
        )
    ),
    inplace=True,
)

In [ ]:
# save data
gpd.GeoDataFrame(
    df,
    crs=unziped_folders[0].crs,
).to_file(filename=output_file.as_posix(), driver="ESRI Shapefile")

# zip data
make_archive(output_path, zipped_output_file)

In [76]:
# clean unzipped files
rm_tree(temp_working_path) if temp_working_path.exists() else None
rm_tree(output_path) if output_path.exists() else None

In [13]:
# LOAD
## load zipped file to GCS
writeReadGCP(
    credentials=mysettings.GCS_KEYFILE_JSON,
    bucket_name=mysettings.GCS_BUCKET,
    blob_name=remote_path,
    file=zipped_output_file,
    operation="w",
)

### Countries gadm intermediate

In [4]:
# Pipe params
force_clean = True
step = "preprocess"
pipe = "gadm"

In [5]:
working_folder = FileConventionHandler(pipe)
input_path = working_folder.pipe_raw_path
temp_working_path = working_folder.get_temp_file_path(step)

output_path = working_folder.get_processed_step_path(step)
output_file = working_folder.get_step_fmt_file_path(step, "shp")
zipped_output_file = working_folder.get_step_fmt_file_path(step, "zip", True)
remote_path = working_folder.get_remote_path(step)

In [4]:
gadm_url = "https://geodata.ucdavis.edu/gadm/gadm4.1/gadm_410-levels.zip"
gadm_file_name = "gadm_410-levels.zip"

In [5]:
# Download data
input_file = downloadFile(
    gadm_url,
    input_path,
    overwrite=force_clean,
    file=gadm_file_name,
)

In [7]:
# Check if there is a zip file in the input_path
zip_file = next(input_path.glob("*.zip"), None)
if zip_file:
    unziped_folder = temp_working_path.joinpath(zip_file.stem)
    print(f"Processing: {unziped_folder}")

    if unziped_folder.exists() and force_clean:
        shutil.rmtree(unziped_folder)
        print(f"Removed existing folder: {unziped_folder}")

    # Unpack the archive
    shutil.unpack_archive(zip_file, unziped_folder)
    print(f"Unpacked {zip_file} to {unziped_folder}")


Processing: /home/sofia/dev/skytruth-30x30/data/data/gadm/raw/temp_preprocess/gadm_410-levels
Removed existing folder: /home/sofia/dev/skytruth-30x30/data/data/gadm/raw/temp_preprocess/gadm_410-levels
Unpacked /home/sofia/dev/skytruth-30x30/data/data/gadm/raw/gadm_410-levels.zip to /home/sofia/dev/skytruth-30x30/data/data/gadm/raw/temp_preprocess/gadm_410-levels


In [8]:
# Select data adm_0, dissolve and save as shp
geopackage_file = next(unziped_folder.rglob("*.gpkg"), None)

if geopackage_file:
    print(f"Found GeoPackage: {geopackage_file}")

    # Specify the layer to read
    layer_name = "ADM_0"
    gdf = gpd.read_file(geopackage_file, layer=layer_name)
    print(f"Selected layer: {layer_name}")   
    
else:
    print("No GeoPackage file found in the unzipped folder.")

Found GeoPackage: /home/sofia/dev/skytruth-30x30/data/data/gadm/raw/temp_preprocess/gadm_410-levels/gadm_410-levels.gpkg
Selected layer: ADM_0


In [25]:
def update_gid_0_and_country(gdf: gpd.GeoDataFrame) -> gpd.GeoDataFrame:
    """
    Updates the GID_0 and COUNTRY values in the GeoDataFrame for dependent territories 
    with the GID_0 and COUNTRY of their sovereign parent countries.

    Parameters:
    gdf (gpd.GeoDataFrame): The input GeoDataFrame with 'GID_0' and 'COUNTRY' columns.

    Returns:
    gpd.GeoDataFrame: The GeoDataFrame with updated 'GID_0' and 'COUNTRY' values for dependent territories.
    """
    # Load the dependency_to_parent mapping
    with open(scripts_dir.joinpath('data_commons/data/dependency_to_parent.json'), 'r') as json_file:
        dependency_to_parent = json.load(json_file)

    # Map GID_0 to the updated values
    gdf['GID_0'] = gdf['GID_0'].map(lambda x: dependency_to_parent.get(x, (x, x))[0])
    
    # Update COUNTRY based on the updated GID_0
    gdf['COUNTRY'] = gdf['GID_0'].map(lambda x: {v[0]: v[1] for k, v in dependency_to_parent.items()}.get(x, gdf['COUNTRY'].loc[gdf['GID_0'] == x].values[0]))

    return gdf


def add_translations(df, translations_csv_path):
    translations_df = pd.read_csv(translations_csv_path, keep_default_na=False, na_values=[])
    
    df = df.merge(translations_df[['code', 'name_es', 'name_fr']], left_on='GID_0', right_on='code', how='left')
    
    return df

In [18]:
# Assign territories to their parent countries
gdf_updated = update_gid_0_and_country(gdf)

# Dissolve by country
gdf_updated = gdf_updated.dissolve(by='COUNTRY').reset_index()

# Calculate area
gdf_updated = gdf_updated.pipe(calculate_area)

In [26]:
# Download country translations
working_folder = FileConventionHandler(pipe)
input_path = working_folder.pipe_raw_path

translations_csv_url = "vizzuality_processed_data/gadm/preprocess/locations_translated.csv"
translations_csv_output = input_path.joinpath(translations_csv_url.split("/")[-1])

writeReadGCP(
    credentials=mysettings.GCS_KEYFILE_JSON,
    bucket_name=mysettings.GCS_BUCKET,
    blob_name=translations_csv_url,
    file=translations_csv_output,
    operation="r",
)

translations_path = input_path.joinpath('locations_translated.csv')

In [27]:
# Add translations for country names
gdf_translated = add_translations(gdf_updated, translations_path).drop(columns=['code'])
gdf_translated

,COUNTRY,geometry,GID_0,area_km2,name_es,name_fr
0,Afghanistan,"MULTIPOLYGON (((63.61425 29.46993, 63.60868 29...",AFG,644050.28,Afganistán,Afghanistan
1,Albania,"MULTIPOLYGON (((19.27804 40.50524, 19.28189 40...",ALB,28689.62,Albania,Albanie
2,Algeria,"MULTIPOLYGON (((2.84535 36.74691, 2.84597 36.7...",DZA,2311455.23,Argelia,Algérie
3,Andorra,"POLYGON ((1.61725 42.62406, 1.63334 42.62553, ...",AND,450.35,Andorra,Andorre
4,Angola,"MULTIPOLYGON (((11.78636 -16.78001, 11.78478 -...",AGO,1251701.39,Angola,Angola
...,...,...,...,...,...,...
199,Vietnam,"MULTIPOLYGON (((103.46895 9.25602, 103.46736 9...",VNM,330364.87,Vietnam,Vietnam
200,Western Sahara,"MULTIPOLYGON (((-16.83569 22.15403, -16.83597 ...",ESH,267892.77,Sahara Occidental,Sahara occidental
201,Yemen,"MULTIPOLYGON (((42.1618 15.03042, 42.16236 15....",YEM,453741.18,Yemen,Yémen
202,Zambia,"POLYGON ((25.87834 -17.97218, 25.87034 -17.970...",ZMB,753990.33,Zambia,Zambie


In [28]:
final_gadm = await simplify_async(gdf_updated)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 204/204 [05:58<00:00,  1.75s/it]


In [29]:
# Save the file
final_gadm.to_file(output_file.as_posix(), driver="ESRI Shapefile")

In [30]:
# zip data
make_archive(output_path, zipped_output_file)

In [31]:
# load zipped file to GCS
writeReadGCP(
    credentials=mysettings.GCS_KEYFILE_JSON,
    bucket_name=mysettings.GCS_BUCKET,
    blob_name=remote_path,
    file=zipped_output_file,
    operation="w",
)

### Mpa Atlas intermediate

In [12]:
force_clean = True
step = "preprocess"
pipe = "mpaatlas"

In [13]:
# Data source
mpaatlas_url = "https://guide.mpatlas.org/api/v1/zone/geojson"
mpaatlas_file_name = "mpatlas_assess_zone.geojson"

In [14]:
working_folder = FileConventionHandler(pipe)
input_path = working_folder.pipe_raw_path
temp_working_path = working_folder.get_temp_file_path(step)

output_path = working_folder.get_processed_step_path(step)
output_file = working_folder.get_step_fmt_file_path(step, "shp")
zipped_output_file = working_folder.get_step_fmt_file_path(step, "zip", True)
remote_path = working_folder.get_remote_path(step)

In [15]:
# Download data
input_file = downloadFile(
    mpaatlas_url,
    input_path,
    overwrite=force_clean,
    file=mpaatlas_file_name,
)

In [16]:
if not force_clean and zipped_output_file.exists():
    print(f"File {zipped_output_file} already exists")

# Transform data
gdf = gpd.read_file(input_file)

df = (gdf
      .pipe(set_wdpa_id)
      .pipe(protection_level)
      .pipe(status)
      .pipe(create_year))

df.drop(
    columns=list(
        set(df.columns)
        - set(
            [
                "wdpa_id",
                "mpa_zone_id", 
                "name",
                "designation",
                "sovereign",
                "establishment_stage",
                "protection_mpaguide_level",
                "protection_level",
                "year",
                "geometry",
            ]
        )
    ),
    inplace=True,
)
df.rename(columns={"sovereign": "location_id", "wdpa_pid": "wdpa_id"}, inplace=True)

In [17]:
#save data
gpd.GeoDataFrame(
    df,
    crs=gdf.crs,
).to_file(filename=output_file.as_posix(), driver="ESRI Shapefile", encoding="utf-8")

make_archive(output_path, zipped_output_file)

/tmp/ipykernel_3376415/3601108936.py:5: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  ).to_file(filename=output_file.as_posix(), driver="ESRI Shapefile", encoding="utf-8")
/home/sofia/miniforge3/envs/skytruth/lib/python3.12/site-packages/pyogrio/raw.py:709: RuntimeWarning: Normalized/laundered field name: 'mpa_zone_id' to 'mpa_zone_i'
  ogr_write(
/home/sofia/miniforge3/envs/skytruth/lib/python3.12/site-packages/pyogrio/raw.py:709: RuntimeWarning: Normalized/laundered field name: 'designation' to 'designatio'
  ogr_write(
/home/sofia/miniforge3/envs/skytruth/lib/python3.12/site-packages/pyogrio/raw.py:709: RuntimeWarning: Normalized/laundered field name: 'location_id' to 'location_i'
  ogr_write(
/home/sofia/miniforge3/envs/skytruth/lib/python3.12/site-packages/pyogrio/raw.py:709: RuntimeWarning: Normalized/laundered field name: 'establishment_stage' to 'establishm'
  ogr_write(
/home/sofia/miniforge3/envs/skytruth/lib/python3.12/

In [10]:
# LOAD
## load zipped file to GCS
writeReadGCP(
    credentials=mysettings.GCS_KEYFILE_JSON,
    bucket_name=mysettings.GCS_BUCKET,
    blob_name=remote_path,
    file=zipped_output_file,
    operation="w",
)

In [13]:
# clean unzipped files
rm_tree(temp_working_path) if temp_working_path.exists() else None
rm_tree(output_path) if output_path.exists() else None

### Protected seas intermediate

In [3]:
# DEPRECATED
force_clean = True
step = "preprocess"
pipe = "protectedseas"

In [24]:
working_folder = FileConventionHandler(pipe)
input_path = working_folder.pipe_raw_path
temp_working_path = working_folder.get_temp_file_path(step)

output_path = working_folder.get_processed_step_path(step)
output_file = working_folder.get_step_fmt_file_path(step, "shp")
zipped_output_file = working_folder.get_step_fmt_file_path(step, "zip", True)
remote_path = working_folder.get_remote_path(step)

In [ ]:
ps_csv_url = "ProtectedSeas/ProtectedSeas_06142023.csv"
ps_csv_output = input_path.joinpath(ps_csv_url.split("/")[-1])

ps_geometries_url = (
    "ProtectedSeas/ProtectedSeas_ProtectedSeas_06142023_shp_ProtectedSeas_06142023_shp.zip"
)
ps_geometries_output = input_path.joinpath(ps_geometries_url.split("/")[-1])

In [25]:
if not force_clean and zipped_output_file.exists():
    print(f"File {zipped_output_file} already exists")

In [26]:
## get the data

writeReadGCP(
    credentials=mysettings.GCS_KEYFILE_JSON,
    bucket_name=mysettings.GCS_BUCKET,
    blob_name=ps_csv_url,
    file=ps_csv_output,
    operation="r",
)

writeReadGCP(
    credentials=mysettings.GCS_KEYFILE_JSON,
    bucket_name=mysettings.GCS_BUCKET,
    blob_name=ps_geometries_url,
    file=ps_geometries_output,
    operation="r",
)

In [27]:
# unzip shapefile
shutil.unpack_archive(ps_geometries_output, temp_working_path)

In [ ]:
# transform data
# TODO: Modify the preprocessing steps so we do not eliminate the geometries that does not intersect with MPAs - do to a change in the processing methodology
data_table = pd.read_csv(ps_csv_output).pipe(get_mpas).pipe(set_location_iso).pipe(set_fps_classes)

data_table.drop(
    columns=data_table.columns.difference(
        [
            "site_id",
            "iso",
            "FPS_cat",
            "site_name",
            "country",
            "wdpa_id",
            "removal_of_marine_life_is_prohibited",
            "total_area",
        ]
    ),
    inplace=True,
)

data_table.rename(columns={"removal_of_marine_life_is_prohibited": "FPS"}, inplace=True)

# load geoemtries & merge

gdf = gpd.read_file(ps_geometries_output)

In [29]:
# save data
gdf.merge(data_table, how="inner", left_on="SITE_ID", right_on="site_id").drop(
    columns=["SITE_ID", "SITE_NAME"]
).to_file(filename=output_file.as_posix(), driver="ESRI Shapefile", encoding="utf-8")

# zip data
make_archive(output_path, zipped_output_file)

In [30]:
# clean unzipped files
rm_tree(temp_working_path) if temp_working_path.exists() else None
rm_tree(output_path) if output_path.exists() else None

In [31]:
# LOAD
## load zipped file to GCS
writeReadGCP(
    credentials=mysettings.GCS_KEYFILE_JSON,
    bucket_name=mysettings.GCS_BUCKET,
    blob_name=remote_path,
    file=zipped_output_file,
    operation="w",
)

### Mpas protected planet intermediate

In [5]:
force_clean = True
step = "preprocess"
pipe = "mpa"

In [6]:
mpa_url = "https://www.protectedplanet.net/downloads"
mpa_body = {
    "domain": "general",
    "format": "shp",
    "token": "marine",
    "id": 21961,
}

In [7]:
working_folder = FileConventionHandler(pipe)
input_path = working_folder.pipe_raw_path
temp_working_path = working_folder.get_temp_file_path(step)

output_path = working_folder.get_processed_step_path(step)
output_file = working_folder.get_step_fmt_file_path(step, "shp")
zipped_output_file = working_folder.get_step_fmt_file_path(step, "zip", True)
remote_path = working_folder.get_remote_path(step)

# download data
r = requests.post(url=mpa_url, data=mpa_body)
r.raise_for_status()

download_url = r.json().get("url")
input_file_name = f'{r.json().get("title")}.zip'
print(r.json())

input_file =  downloadFile(
    url=download_url,
    output_path=input_path,
    overwrite=force_clean,
    file=input_file_name,
)

{'id': 'marine-shp', 'title': 'WDPA_WDOECM_Sep2024_Public_marine_shp', 'url': 'https://d1gam3xoknrgr2.cloudfront.net/current/WDPA_WDOECM_Sep2024_Public_marine_shp.zip', 'hasFailed': False, 'token': 'marine'}


In [8]:
# unzip file twice due how data is provisioned by protected planet
shutil.unpack_archive(
    input_file,
    temp_working_path,
    "zip",
)

for file in temp_working_path.glob("*.zip"):
    shutil.unpack_archive(file, temp_working_path.joinpath(file.stem), "zip")

In [9]:
# load data & Transform it
unziped_folders = []
for file in temp_working_path.glob("*/*.shp"):
    df = (
        gpd.read_file(file)
        .pipe(filter_by_methodology)
        .pipe(transform_points)
        .pipe(clean_geometries)
    )
    unziped_folders.append(df)

# merge datasets
gdf = gpd.GeoDataFrame(
    pd.concat(unziped_folders, ignore_index=True),
    crs=unziped_folders[0].crs,
)

gdf.drop(
    columns=list(
        set(gdf.columns)
        - set(
            [
                "geometry",
                "WDPAID",
                "WDPA_PID",
                "PA_DEF",
                "NAME",
                "PARENT_ISO",
                "DESIG_ENG",
                "IUCN_CAT",
                "STATUS",
                "STATUS_YR",
                "GIS_M_AREA",
                "AREA_KM2",
            ]
        )
    ),
    inplace=True,
)
gdf["WDPAID"] = pd.to_numeric(gdf["WDPAID"], downcast="integer")

In [10]:
# save data & zip it
gdf.to_file(filename=output_file, driver="ESRI Shapefile", encoding="utf-8")

make_archive(output_path, zipped_output_file)

In [18]:
# LOAD
## load zipped file to GCS
writeReadGCP(
    credentials=mysettings.GCS_KEYFILE_JSON,
    bucket_name=mysettings.GCS_BUCKET,
    blob_name=remote_path,
    file=zipped_output_file,
    operation="w",
)

In [11]:
remote_path

'vizzuality_processed_data/mpa/preprocess/mpa_preprocess.zip'

In [19]:
# clean unzipped files
rm_tree(temp_working_path) if temp_working_path.exists() else None
rm_tree(output_path) if output_path.exists() else None

### Pas protected planet intermediate terrestrial

In [37]:
force_clean = True
step = "preprocess"
pipe = "mpa-terrestrial"

In [38]:
mpa_url = "https://www.protectedplanet.net/downloads"
mpa_body = {
    "domain": "general",
    "format": "shp",
    "token": "wdpa",
    "id": 76011,
}

In [39]:
working_folder = FileConventionHandler(pipe)
input_path = working_folder.pipe_raw_path
temp_working_path = working_folder.get_temp_file_path(step)

output_path = working_folder.get_processed_step_path(step)
output_file = working_folder.get_step_fmt_file_path(step, "gpkg")
zipped_output_file = working_folder.get_step_fmt_file_path(step, "zip", True)
remote_path = working_folder.get_remote_path(step)

In [ ]:
# download data
r = requests.post(url=mpa_url, data=mpa_body)
r.raise_for_status()

download_url = r.json().get("url")
input_file_name = f'{r.json().get("title")}.zip'
print(r.json())

input_file = downloadFile(
    url=download_url,
    output_path=input_path,
    overwrite=force_clean,
    file=input_file_name,
)

In [15]:
# unzip file twice due how data is provisioned by protected planet
shutil.unpack_archive(
    input_file,
    temp_working_path,
    "zip",
)

for file in temp_working_path.glob("*.zip"):
    shutil.unpack_archive(file, temp_working_path.joinpath(file.stem), "zip")

In [16]:
# load data & Transform it
unziped_folders = []
for file in temp_working_path.glob("*/*.shp"):
    df = (
        gpd.read_file(file)
        .pipe(filter_by_methodology)
        .pipe(filter_by_terrestrial)
        .pipe(transform_points)
        .pipe(clean_geometries)
    )
    unziped_folders.append(df)

# merge datasets
gdf = gpd.GeoDataFrame(
    pd.concat(unziped_folders, ignore_index=True),
    crs=unziped_folders[0].crs,
)

gdf.drop(
    columns=list(
        set(gdf.columns)
        - set(
            [
                "geometry",
                "WDPAID",
                "WDPA_PID",
                "PA_DEF",
                "NAME",
                "PARENT_ISO",
                "DESIG_ENG",
                "IUCN_CAT",
                "STATUS",
                "STATUS_YR",
                "GIS_AREA",
                "MARINE",
            ]
        )
    ),
    inplace=True,
)
gdf["WDPAID"] = pd.to_numeric(gdf["WDPAID"], downcast="integer")

In [17]:
final_wdpa_terrestrial = await simplify_async(gdf)

  0%|                                                                                                                                                    | 85/292261 [00:00<14:27, 336.92it/s]

<class 'shapely.geometry.base.GeometrySequence'>


  0%|▎                                                                                                                                               | 661/292261 [00:07<145:23:41,  1.79s/it]

<class 'shapely.geometry.base.GeometrySequence'>


  1%|██                                                                                                                                               | 4100/292261 [00:10<00:50, 5746.09it/s]

...

 95%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏      | 278285/292261 [01:04<00:02, 6507.33it/s]

<class 'shapely.geometry.base.GeometrySequence'>
<class 'shapely.geometry.base.GeometrySequence'>


 99%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉ | 290011/292261 [01:11<00:00, 5892.57it/s]

<class 'shapely.geometry.base.GeometrySequence'>


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 292261/292261 [03:13<00:00, 1507.66it/s]


In [18]:
# save data & zip it
final_wdpa_terrestrial.to_file(
    filename=output_file,
    driver="GPKG",
    layer="name",
    encoding="utf-8",
)

In [19]:
output_file

PosixPath('/home/sofia/dev/skytruth-30x30/data/data/mpa-terrestrial/processed/preprocess/mpa-terrestrial_preprocess.gpkg')

In [41]:
final_wdpa_terrestrial['MARINE'].unique()

array(['0', '1'], dtype=object)

In [ ]:
# LOAD
## load zipped file to GCS
writeReadGCP(
    credentials=mysettings.GCS_KEYFILE_JSON,
    bucket_name=mysettings.GCS_BUCKET,
    blob_name=remote_path,
    file=output_file,
    operation="w",
)

In [ ]:
# clean unzipped files
rm_tree(temp_working_path) if temp_working_path.exists() else None
rm_tree(output_path) if output_path.exists() else None

### Protected planet intermediate all

In [43]:
force_clean = True
step = "preprocess"
pipe = "pa"

In [44]:
mpa_url = "https://www.protectedplanet.net/downloads"
mpa_body = {
    "domain": "general",
    "format": "shp",
    "token": "wdpa",
    "id": 76011,
}

In [45]:
working_folder = FileConventionHandler(pipe)
input_path = working_folder.pipe_raw_path
temp_working_path = working_folder.get_temp_file_path(step)

output_path = working_folder.get_processed_step_path(step)
output_file = working_folder.get_step_fmt_file_path(step, "gpkg")
zipped_output_file = working_folder.get_step_fmt_file_path(step, "zip", True)
remote_path = working_folder.get_remote_path(step)

In [46]:
# download data
r = requests.post(url=mpa_url, data=mpa_body)
r.raise_for_status()

download_url = r.json().get("url")
input_file_name = f'{r.json().get("title")}.zip'
print(r.json())

# input_file = downloadFile(
#     url=download_url,
#     output_path=input_path,
#     overwrite=force_clean,
#     file=input_file_name,
# )

{'id': 'wdpa-shp', 'title': 'WDPA_Sep2024_Public_shp', 'url': 'https://d1gam3xoknrgr2.cloudfront.net/current/WDPA_Sep2024_Public_shp.zip', 'hasFailed': False, 'token': 'wdpa'}


In [47]:
# unzip file twice due how data is provisioned by protected planet
shutil.unpack_archive(
    input_file,
    temp_working_path,
    "zip",
)

for file in temp_working_path.glob("*.zip"):
    shutil.unpack_archive(file, temp_working_path.joinpath(file.stem), "zip")

In [68]:
# load data & Transform it
unziped_folders = []
for file in temp_working_path.glob("*/*.shp"):
    df = (
        gpd.read_file(file)
        .pipe(filter_by_methodology)
        .pipe(transform_points)
        .pipe(clean_geometries)
    )
    unziped_folders.append(df)

# merge datasets
gdf = gpd.GeoDataFrame(
    pd.concat(unziped_folders, ignore_index=True),
    crs=unziped_folders[0].crs,
)

gdf.drop(
    columns=list(
        set(gdf.columns)
        - set(
            [
                "geometry",
                "WDPAID",
                "WDPA_PID",
                "PA_DEF",
                "NAME",
                "PARENT_ISO",
                "DESIG_ENG",
                "IUCN_CAT",
                "STATUS",
                "STATUS_YR",
                "GIS_AREA",
                "GIS_M_AREA",
                "MARINE",
            ]
        )
    ),
    inplace=True,
)
gdf["WDPAID"] = pd.to_numeric(gdf["WDPAID"], downcast="integer")

In [70]:
final_wdpa = await simplify_async(gdf)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 298912/298912 [03:53<00:00, 1277.78it/s]


<class 'shapely.geometry.base.GeometrySequence'>


  1%|▉                                                                                                                                                            | 1817/298912 [00:11<03:42, 1338.09it/s]

<class 'shapely.geometry.base.GeometrySequence'>


  1%|█▉                                                                                                                                                           | 3731/298912 [00:12<03:25, 1433.85it/s]

...

 95%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎       | 284183/298912 [01:23<00:04, 3294.97it/s]

<class 'shapely.geometry.base.GeometrySequence'>


 99%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌ | 296106/298912 [01:32<00:00, 3532.30it/s]

<class 'shapely.geometry.base.GeometrySequence'>


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 298912/298912 [03:53<00:00,  2.59it/s]

In [71]:
# save data & zip it
final_wdpa.to_file(
    filename=output_file,
    driver="GPKG",
    layer="name",
    encoding="utf-8",
)

In [75]:
# LOAD
## load zipped file to GCS
writeReadGCP(
    credentials=mysettings.GCS_KEYFILE_JSON,
    bucket_name=mysettings.GCS_BUCKET,
    blob_name=remote_path,
    file=output_file,
    operation="w",
)

In [ ]:
# clean unzipped files
rm_tree(temp_working_path) if temp_working_path.exists() else None
rm_tree(output_path) if output_path.exists() else None

### Habitats

In [4]:
force_clean = True
step = "preprocess"
pipe = "habitats"

In [5]:
habitats_download_url = "https://habitats.oceanplus.org/downloads/global_statistics.zip"
Mangroves_download_url = "https://mangrove-atlas-api.herokuapp.com/admin/widget_protected_areas.csv"
mangroves_request_headers = {
    "Cookie": "_mangrove_atlas_api_session=fJuobvI2fH42WfGfMtRTp%2BksIDdPEpY6DG8uCuITsENtrRGG4AA3nYEeAI7dytzpK%2F0dGIHq84O54MRr6eiPgiwCYXp2XP4IzXM40dFt%2FI6hoB0WXC%2Fwrd81XreNnMZiSEE6IVT5R0fqMcmsZdPn53u0A1d4CGU3FfliOZuWkckBuA%2F7C4upBGuSS8817LqOh1slG%2BsEOGp3nk7WX4fMoPbsHWtARfFwdfoAHz448LO7uWuZdyiu7YOrS0ZxOZEb9JZ8hcUJph4pBFofZLpOvtQQutgZY21T5bhQ7Kwfl56e6Qr0SZ%2B8sIzMfky3h%2FjOA6DNTLoy%2BZLiZBAgFHlTYm2JwlwqWgAZU8D7cE7Zn%2Fxgf3LFF9pZ9Fe3QG4c8LIwH%2FxqjEd8GsZAhBMgBWbxubigQ9gZssZt6CIO--7qiVsTAT8JAKj1jU--U7TI%2Fz9c151bfD8iZdkBDw%3D%3D"
}
seamounts_download_url = "https://datadownload-production.s3.amazonaws.com/ZSL002_ModelledSeamounts2011_v1.zip"

In [6]:
working_folder = FileConventionHandler(pipe)
input_path = working_folder.pipe_raw_path
temp_working_path = working_folder.get_temp_file_path(step)

output_path = working_folder.get_processed_step_path(step)
output_file = working_folder.get_step_fmt_file_path(step, "shp")
zipped_output_file = working_folder.get_step_fmt_file_path(step, "zip", True)
remote_path = working_folder.get_remote_path(step)

#### Seamounts

In [8]:
input_seamounts_path = input_path.joinpath("seamounts")
input_seamounts_path.mkdir(parents=True, exist_ok=True)
# download data
input_file_name = "seamounts.zip"
input_file = downloadFile(
    url=seamounts_download_url,
    output_path=input_seamounts_path,
    overwrite=force_clean,
    file=input_file_name,
)

In [9]:
# unzip data
shutil.unpack_archive(
    input_file,
    temp_working_path,
    "zip",
)

In [ ]:
temp_working_path

In [24]:
first =gpd.read_file(next(temp_working_path.rglob("*SeamountsBaseArea.shp")))

In [ ]:
first

In [ ]:
if not force_clean and zipped_output_file.exists():
    print(f"File {zipped_output_file} already exists")